In [14]:
import pandas as pd
df=messages = pd.read_csv('spam.csv',encoding='latin-1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [11]:
import numpy as np
np.bool = np.bool_
np.int = np.int_
np.object = np.object_

In [12]:
from tensorflow import keras
from tensorflow.python import traceback

In [3]:
#Let's drop the non-relevant unnamed columns
df=df.drop(['Unnamed: 2', 'Unnamed: 3','Unnamed: 4'],axis=1)

# Renaming v1 & v2 as Category & Text
df=df.rename(columns={"v1":"Category","v2":"Text"})

#Sample post modifications
df.head()

,Category,Text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
df = df.rename(columns={'Category': 'label', 'Text': 'message'})

In [5]:
y=list(df['label'])
y=list(pd.get_dummies(y,drop_first=True)['spam'])

In [6]:
df['label'] = y

In [7]:
df.head()

,label,message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [15]:
import pandas as pd
from transformers import BertTokenizer, TFBertForSequenceClassification
import tensorflow as tf
from sklearn.model_selection import train_test_split  # Import train_test_split
import numpy as np # Import numpy for label handling

# Sample DataFrame (replace with your actual data)
data = {'message': ["This is a spam email.", "Meeting reminder.", "Get rich quick!", "Important update.", "Free offer!"],
        'label': [True, False, True, False, True]}
df = pd.DataFrame(data)

# Prepare the data
X = list(df['message'])
y = list(df['label'])

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tokenize the text
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_encodings = tokenizer(X_train, truncation=True, padding=True)
test_encodings = tokenizer(X_test, truncation=True, padding=True)

# Convert boolean labels to integers (0 and 1)
y_train = np.array(y_train).astype(int)  # Crucial for TensorFlow
y_test = np.array(y_test).astype(int)

# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train
)).shuffle(100).batch(16) # Shuffle and batch here

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    y_test
)).batch(16) # Batch here


# Load pre-trained BERT model
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)  # Important: from_logits=True
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

# Train the model
model.fit(train_dataset, epochs=3)  # No need to specify batch_size again

# Evaluate the model
loss, accuracy = model.evaluate(test_dataset)
print(f"Loss: {loss}")
print(f"Accuracy: {accuracy}")


# Example prediction (after training)
new_texts = ["This is a new spam message.", "Another normal email."]
new_encodings = tokenizer(new_texts, truncation=True, padding=True)
new_dataset = tf.data.Dataset.from_tensor_slices(dict(new_encodings)).batch(16)
predictions = model.predict(new_dataset)

# Get predicted labels (0 or 1)
predicted_labels = tf.argmax(predictions.logits, axis=1).numpy()
print("Predicted labels:", predicted_labels)

# Convert back to True/False if needed
predicted_bool = [bool(label) for label in predicted_labels]
print("Predicted labels (boolean):", predicted_bool)

RuntimeError: Failed to import transformers.models.bert.modeling_tf_bert because of the following error (look up to see its traceback):
DLL load failed while importing _pywrap_dtensor_device: The specified procedure could not be found.

In [ ]:
# prompt: how to check using random text input here?

import numpy as np

def predict_spam(text, model, tokenizer):
    inputs = tokenizer(text, truncation=True, padding=True, return_tensors='tf')
    outputs = model(inputs)
    logits = outputs.logits
    probabilities = tf.nn.softmax(logits, axis=-1).numpy()[0]
    predicted_class = np.argmax(probabilities)
    return predicted_class, probabilities

# Example usage (assuming 'model' and 'tokenizer' are already defined):
random_text = "Congratulations! You've won a free cruise."
predicted_class, probabilities = predict_spam(random_text, model, tokenizer)

print(f"Predicted class: {predicted_class}") # 1 for spam, 0 for not spam
print(f"Probabilities: {probabilities}")

Predicted class: 1
Probabilities: [0.08317112 0.9168289 ]


In [ ]:
# prompt: save this model as h5, pickle file, json

import json
import pickle

# Save as HDF5
model.save_weights('my_model.h5')

# Save as pickle
with open('my_model.pkl', 'wb') as f:
    pickle.dump(model, f)

# Save model architecture and weights as JSON (not the entire model, but its configuration)
model_config = model.to_json()
with open('my_model_architecture.json', 'w') as json_file:
    json.dump(model_config, json_file)

# Save tokenizer (important for later use with the saved model)
tokenizer.save_pretrained('my_tokenizer')


/usr/local/lib/python3.11/dist-packages/transformers/generation/tf_utils.py:465: UserWarning: `seed_generator` is deprecated and will be removed in a future version.
  warnings.warn("`seed_generator` is deprecated and will be removed in a future version.", UserWarning)


('my_tokenizer/tokenizer_config.json',
 'my_tokenizer/special_tokens_map.json',
 'my_tokenizer/vocab.txt',
 'my_tokenizer/added_tokens.json')

In [ ]:
!ls

my_model_architecture.json  my_model.h5  my_model.pkl  my_tokenizer  sample_data  spam.csv


In [ ]:
# prompt: download all the files currently available

from google.colab import files
import os

for filename in os.listdir():
  if not filename.startswith('.'): #ignore hidden files
    files.download(filename)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# prompt: how to run this model using flask

import flask
from flask import Flask, request, jsonify
import tensorflow as tf
from transformers import BertTokenizer
import numpy as np
import pickle
import json
import os


app = Flask(__name__)

# Load the saved model and tokenizer
# You need to ensure these files are in the same directory as your app.py
# or provide the correct paths


model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
model.load_weights('my_model.h5')

with open('my_model_architecture.json', 'r') as json_file:
    model_config = json.load(json_file)

tokenizer = BertTokenizer.from_pretrained('my_tokenizer')

def predict_spam(text, model, tokenizer):
    inputs = tokenizer(text, truncation=True, padding=True, return_tensors='tf')
    outputs = model(inputs)
    logits = outputs.logits
    probabilities = tf.nn.softmax(logits, axis=-1).numpy()[0]
    predicted_class = np.argmax(probabilities)
    return predicted_class, probabilities

@app.route('/predict', methods=['POST'])
def predict():
    try:
        data = request.get_json()
        text = data['text']

        predicted_class, probabilities = predict_spam(text, model, tokenizer)

        response = {
            'predicted_class': int(predicted_class),  # Convert to integer
            'probabilities': probabilities.tolist()   # Convert to list
        }

        return jsonify(response)
    except Exception as e:
        return jsonify({'error': str(e)}), 500


if __name__ == '__main__':
    app.run(debug=True, host='0.0.0.0', port=int(os.environ.get("PORT", 8080)))


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


 * Serving Flask app '__main__'
 * Debug mode: on


Address already in use
Port 8080 is in use by another program. Either identify and stop that program, or start the server with a different port.
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/serving.py", line 759, in __init__
    self.server_bind()
  File "/usr/lib/python3.11/http/server.py", line 136, in server_bind
    socketserver.TCPServer.server_bind(self)
  File "/usr/lib/python3.11/socketserver.py", line 472, in server_bind
    self.socket.bind(self.server_address)
OSError: [Errno 98] Address already in use

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-12-1d703bc33a19>", line 55, in <cell line: 0>
    app.run(debug=True, host='0.0.0.0', port=int(os.environ.get("PORT", 8080)))
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 662, in run
    run_simple(t.cast(str, host), port, self, **options)
  File "/usr/local/lib/pytho

TypeError: object of type 'NoneType' has no len()